Notebook to generate prediction model using features available in FHIR server
---------------------------------------------------------------------------------
<br>
Item IDs used for prediction - 220052, 22045, 224690, 223761, 220277,  227013

In [1]:
import sklearn 
import numpy as np 
import pandas as pd 
from datetime import datetime

In [2]:
def load_data(path2, path4):
    #df_icu_stays = pd.read_csv(path1)
    df_sepsis_occurance = pd.read_csv(path2)
    #df_events = pd.read_csv(path3)
    df_chart_events = pd.read_csv(path4)
    return df_sepsis_occurance, df_chart_events

path1 = 'm100s2/ICUSTAYS.csv'
path2 = 'm100s2/sample_ids_2.csv'
path3 = 'm100s2/sample_inputevents_mv_2.csv'
path4 = "m100s2/sample_chartevents_2.csv"
df_sepsis_occurance, df_chart_events = load_data(path2, path4)
#df_icu_stays, df_sepsis_occurance, df_events, df_chart_events = load_data(path1,path2,path3,path4)

/Users/jyon/Tools/anaconda3/envs/tensorflow/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2821: DtypeWarning: Columns (8,10,13,14) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


In [3]:
print(df_sepsis_occurance.shape, df_chart_events.shape)

((100, 4), (3168992, 15))


In [4]:
print(df_chart_events.columns.values)

['row_id' 'subject_id' 'hadm_id' 'icustay_id' 'itemid' 'charttime'
 'storetime' 'cgid' 'value' 'valuenum' 'valueuom' 'warning' 'error'
 'resultstatus' 'stopped']


In [5]:
def join_data(df_sepsis_occurance, df_chart_events): 
    df1 = pd.merge(df_chart_events, df_sepsis_occurance, on='hadm_id', how='inner')
    #df1 = pd.merge(df, df_chart_events, on='hadm_id', how='inner')
    return df1

df_joined = join_data(df_sepsis_occurance, df_chart_events)

In [6]:
print(df_joined.columns.values)

['row_id' 'subject_id_x' 'hadm_id' 'icustay_id_x' 'itemid' 'charttime'
 'storetime' 'cgid' 'value' 'valuenum' 'valueuom' 'warning' 'error'
 'resultstatus' 'stopped' 'subject_id_y' 'icustay_id_y' 'sepsist0']


In [7]:
all_vals = df_joined['itemid'].values
ids = [220052, 22045, 224690, 223761, 220277,  227013]
for i in ids:
    print(len(all_vals[all_vals == i]))
[220052, 224690, 223761, 220277]

37680
0
44166
11769
82788
0


In [8]:
df_joined.shape

(3372162, 18)

In [9]:
df_joined = df_joined[df_joined['itemid'].isin(ids)]

In [10]:
df_joined.shape

(176403, 18)

In [11]:
#s1 = df_joined[(df_joined.hadm_id == 109151) & (df_joined.itemid == 220277)]
df_joined.columns.values

array(['row_id', 'subject_id_x', 'hadm_id', 'icustay_id_x', 'itemid',
       'charttime', 'storetime', 'cgid', 'value', 'valuenum', 'valueuom',
       'warning', 'error', 'resultstatus', 'stopped', 'subject_id_y',
       'icustay_id_y', 'sepsist0'], dtype=object)

In [64]:
#s1.columns.values
#s1 = pd.Series(s1.charttime)
#s1.sort_values(inplace=True)

In [65]:
#s1

4518    2141-06-08 20:30:00
4524    2141-06-08 21:00:00
4540    2141-06-08 22:00:00
4548    2141-06-08 23:00:00
4779    2141-06-09 00:00:00
6268    2141-06-09 01:00:00
6275    2141-06-09 02:00:00
6281    2141-06-09 03:00:00
6316    2141-06-09 04:00:00
8066    2141-06-09 05:00:00
8072    2141-06-09 06:00:00
8078    2141-06-09 07:00:00
8084    2141-06-09 08:00:00
8097    2141-06-09 09:00:00
8106    2141-06-09 10:00:00
8112    2141-06-09 11:00:00
3818    2141-06-09 12:47:00
3826    2141-06-09 13:00:00
4233    2141-06-09 14:00:00
4239    2141-06-09 15:00:00
4245    2141-06-09 16:00:00
5435    2141-06-09 17:00:00
5452    2141-06-09 18:00:00
5458    2141-06-09 19:00:00
5464    2141-06-09 20:00:00
7416    2141-06-09 22:30:00
7418    2141-06-09 23:00:00
7421    2141-06-10 00:00:00
3379    2141-06-10 01:00:00
3385    2141-06-10 02:00:00
               ...         
6059    2141-06-27 10:07:00
6064    2141-06-27 11:00:00
6067    2141-06-27 12:00:00
6081    2141-06-27 13:00:00
6086    2141-06-27 1

In [12]:
#Convert columns to datetime objects 
df_joined['sepsist0'] = df_joined['sepsist0'].apply(lambda i: datetime.strptime(i,'%Y-%m-%d %H:%M:%S'))
#df_joined['starttime'] = df_joined['starttime'].apply(lambda i: datetime.strptime(i,'%Y-%m-%d %H:%M:%S'))
df_joined['charttime'] = df_joined['charttime'].apply(lambda i: datetime.strptime(i,'%Y-%m-%d %H:%M:%S'))

In [13]:
#Create time difference in hours column between sepsis t0 and starttime
df_joined['time_diff'] = (df_joined['sepsist0'] - df_joined['charttime'])
df_joined['time_diff'] = df_joined['time_diff'].apply(lambda i: i.seconds/3600)

In [14]:
print(df_joined.iloc[12])

row_id                      3846287
subject_id_x                  29030
hadm_id                      109151
icustay_id_x                 272418
itemid                       220052
charttime       2141-06-16 15:00:00
storetime       2141-06-16 15:34:00
cgid                          15047
value                            68
valuenum                         68
valueuom                       mmHg
warning                           0
error                             0
resultstatus                    NaN
stopped                         NaN
subject_id_y                  29030
icustay_id_y                 272418
sepsist0        2141-06-08 00:00:00
time_diff                         9
Name: 1272, dtype: object


# Set Window Size

In [15]:
hours = 5
#1 if developed sepsis with 8 hours else 0
df_joined['curr_sepsis'] = df_joined['time_diff'].apply(lambda i: 1 if i <= hours else 0)

In [16]:
#Number of events with sepsis and without sepsis
print(len(df_joined['curr_sepsis'][df_joined['curr_sepsis'] == 1].values))
print(len(df_joined['curr_sepsis'][df_joined['curr_sepsis'] == 0].values))

44845
131558


# Feature Selection

In [31]:
#Here we select our features and drop NaNs
def filter_features(df,features,pred_label):
    all_feats = features+pred_label
    new_df = df[all_feats]
    new_df = new_df.dropna(axis=0)
    new_x = new_df[features]
    new_y = new_df[pred_label]
    return new_x, new_y 

#features = ['amount','rate','patientweight','totalamount','originalamount','originalrate']
features = ['value']
pred_label = ['curr_sepsis']
filtered_x, filtered_y = filter_features(df_joined,features,pred_label)

<center><h4>Rate Feature</h4></center>

In [17]:
#Creating the sparse rate feature vectors
all_items = df_joined['itemid'].values
#all_rates = df_joined['rate'].values
all_rates = df_joined['value'].values
#all_rates.shape
print(all_items.shape[0])

176403


In [18]:
#Creating the sparse rate feature vectors
all_items = df_joined['itemid'].values
#all_rates = df_joined['rate'].values
all_rates = df_joined['value'].values
all_y = list(df_joined['curr_sepsis'])
item_ids = list(set(df_joined['itemid'].values)) 
remap_items = range(len(item_ids))
dict_items = dict(zip(item_ids,remap_items))
#print(dict_items)
item_ids = list(set(df_joined['itemid'].values)) 
item_id_feature = np.zeros((df_joined.shape[0],len(item_ids)))
x = []
y = []
#print(all_y.shape)
#print(all_y[0])
for i in range(all_items.shape[0]):
    val = all_items[i]
    rate = all_rates[i]
    remap_val = dict_items[val]
    z = np.zeros((1,len(item_ids)))[0]
    z[remap_val] = rate
    x.append(z)
    y.append(all_y[i])
    
rate_feat_x = np.array(x)
y = np.array(y)
print(y.shape)
# for i in range(all_items.shape[0]): 
#     val = all_items[i]
#     rate = all_rates[i]
#     #print(val)
#     remap_val = dict_items[val]
#     z = np.zeros((1,len(item_ids)))[0]
#     z[remap_val] = rate
#     x.append(z)
#     y.append(all_y[i])
# 
# 
# print(y.shape)

#Removing rows with NaNs
nan_index = np.argwhere(np.isnan(rate_feat_x))
nan_rows = []
for i in nan_index: 
    nan_rows.append(i[0])
rate_feat_x = np.delete(rate_feat_x,nan_rows,0)
rate_feat_y = np.delete(y,nan_rows,0)

(176403,)


In [19]:
#Create train/test split
from sklearn.cross_validation import train_test_split
#xtrain, xtest, ytrain, ytest = train_test_split(filtered_x, filtered_y, test_size=0.20)

#Train/Test split for rate sparse feature vector
Rxtrain, Rxtest, Rytrain, Rytest = train_test_split(rate_feat_x, rate_feat_y, test_size=0.20)

/Users/jyon/Tools/anaconda3/envs/tensorflow/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [20]:
print(Rxtrain)

[[   0.    0.   72.    0.]
 [   0.    0.    0.  100.]
 [   0.   24.    0.    0.]
 ..., 
 [   0.    0.    0.   96.]
 [   0.    0.    0.  100.]
 [   0.    0.    0.   99.]]


<center><h4>Building A Random Forest Model</h4></center>

In [54]:
#Create Random Forest Model 

rf = RandomForestClassifier(n_estimators= 25, max_depth= None,random_state= 11 )
rf.fit(xtrain, ytrain)
prediction = rf.predict(xtest)

/Users/jyon/Tools/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:4: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().


In [55]:
#Mean Squared Error of Random Forest
from sklearn.metrics import mean_squared_error
print(mean_squared_error(ytest,prediction))

0.223406366033


In [56]:
#Accuracy of Random Forest
from sklearn.metrics import accuracy_score 
print(accuracy_score(ytest,prediction))

0.776593633967


In [59]:
from sklearn.externals import joblib
# now you can save it to a file
joblib.dump(rf, 'random_forests_CE.pkl') 
# and later you can load it
rf = joblib.load('random_forests_CE.pkl')

<h4>Rate Sparse Feature:</h4>

In [21]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score 
from sklearn.externals import joblib
rf = RandomForestClassifier(n_estimators= 25, max_depth= None,random_state= 11 )
rf.fit(Rxtrain, Rytrain)
Rprediction = rf.predict(Rxtest)
print(accuracy_score(Rytest,Rprediction))
# now you can save it to a file
joblib.dump(rf, 'random_forests_CE.pkl') 
# and later you can load it
rf = joblib.load('random_forests_CE.pkl')

0.745642130325


<center><h4>Building A Logistic Regression Model</h4></center>

In [17]:
from sklearn.linear_model import LogisticRegression 
lr = LogisticRegression(C=10) 
lr.fit(xtrain, ytrain)
predictions = lr.predict(xtest)

/Users/Adamlieberman/anaconda/envs/cse6240hw1/lib/python3.5/site-packages/sklearn/utils/validation.py:526: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [18]:
print(mean_squared_error(ytest,predictions))
print(accuracy_score(ytest,predictions))

0.216813549986
0.783186450014


In [19]:
from sklearn.externals import joblib
# now you can save it to a file
joblib.dump(rf, 'logistic_regression.pkl') 
# and later you can load it
lr = joblib.load('logistic_regression.pkl')